In [47]:
import numpy as np
class connect4:

    def __init__(self, width, height, onMove=1):
        self.width = width
        self.height = height
        self.onMove = onMove
        self.field = np.zeros((height, width), dtype=np.int32) # 0: empty, 1,2: player 1,2
                                                               # heighest row is 0

    def play(self, col):
        assert self.field[0,col] == 0, "col " + str(col) + " already full"
        for h in range(self.height-1, -1, -1):
            if self.field[h, col] == 0:
                self.field[h,col] = self.onMove
                break
        self.onMove = 3-self.onMove
        self.checkWinner((h, col))

    def show(self):
        return self.field
    
    def reset(self):
        self.field = np.zeros((self.height, self.width), dtype=np.int32)
        
    def checkWinner(self, position):
        directions = [(1,0), (1,1), (0,1), (-1, 1)]
        for d in directions:
            counter = 1
            cur_pos = self.mapping(position,d)
            while cur_pos is not None and self.field[cur_pos] == self.field[position]:
                cur_pos = self.mapping(cur_pos, d)
                counter += 1

            d_ = tuple((i*-1 for i in d))
            cur_pos = self.mapping(position,d_)
            while cur_pos is not None and self.field[cur_pos] == self.field[position]:
                cur_pos = self.mapping(cur_pos, d_)
                counter += 1

            if counter >= 4:
                return True
        return False


    def mapping(self, position, move):
        new_pos =  position[0] + move[0], position[1] + move[1]
        if new_pos[0] < 0 or new_pos[1] < 0 or new_pos[0] >= self.height or new_pos[1] >= self.width:
            return None
        else:
            return new_pos
c4 = connect4(5,4)

print(c4.checkWinner((0,2)))
print(c4.show())

True
[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]


In [64]:
import numpy as np
import keras
import main
from keras.layers.core import Dense


class network:
    def __init__(self, height,  width, hiddens=(20, 20)):
        self.model = keras.models.Sequential()
        self.in_dim = width * height + width
        self.hiddens = hiddens
        self.model.add(Dense(hiddens[0], input_dim=self.in_dim)) #input_shape=(self.in_dim,), activation="sigmoid"))
        for i in range(1,len(hiddens)):
            self.model.add(Dense(hiddens[i], activation="sigmoid"))
        self.model.add(Dense(1, activation="sigmoid"))
        self.model.compile(loss="mse", metrics=["accuracy"], optimizer="rmsprop")

    def eval(self, c4: main.connect4, col):
        input = self.formatInput(c4, col)
        output = self.model.predict(input)
        return output


    def train(self, c4):
        pass
    
    def formatInput(self, c4: main.connect4, col: int):
        field = np.copy(c4.field)
        one_hot = np.zeros((1, c4.width))
        one_hot[0,col] = 1
        reshaped = np.reshape(field, (1, -1))
        formatted = np.stack((reshaped, one_hot), axis=1)
        return formatted

In [65]:
network = network(c4.height, c4.width)

In [66]:
print(c4.onMove)
c4.reset()
print(c4.onMove)
c4.play(0)
c4.play(0)
c4.play(1)
c4.play(1)
c4.play(2)
c4.play(2)
c4.play(3)
# c4.field = np.array([[0,0,0,1,0],
#                      [0,0,1,0,0],
#                      [0,1,0,0,0],
#                      [1,0,0,0,0]])
print(c4.show())
print(network.formatInput(c4,1))

1
1
[[0 0 0 0 0]
 [0 0 0 0 0]
 [2 2 2 0 0]
 [1 1 1 1 0]]


ValueError: all input arrays must have the same shape